In [4]:
import pandas as pd
import numpy as np
import numpy.random as npr

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [5]:
#create a list of integrer numbers and then save it to a text file named data_int.txt.
#Run the cat command to print the content of the file.
list1 = [0,41,32,53,94,75,86,27,18,90] #init an intergers list
with open('data_int.txt', mode = 'w') as f:
    for i in list1:
        (f, i, fmt='%.2f')
        
!cat "data_int.txt"


SyntaxError: invalid syntax (2248273536.py, line 6)

In [6]:
#create a matrix of 5x5 floats and then save it to a text file named data_float.txt. 
#Use the cat command to print the content of the file. 

A = np.matrix(npr.uniform(1, 5, size=(5, 5)))
print('A 5x5 matrix: \n',A)
with open('data_float.txt', mode = 'w') as f:
    for i in A:
        np.savetxt(f, i, fmt='%.002f')
        
!cat "data_float.txt"


A 5x5 matrix: 
 [[4.14810438 2.19615953 3.64765159 3.60705253 3.2652039 ]
 [1.20656307 4.730747   3.49985456 4.35669924 4.29929103]
 [1.24162838 3.98411467 2.19189953 4.81679752 2.95989708]
 [3.29499512 3.61645073 1.65745052 4.67585293 2.44945262]
 [4.45909049 4.5381553  4.96676006 4.53154072 1.52908724]]


'cat' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
#Load the txt file of the previous point and convert it to a csv file by hand.
import csv 

df = pd.read_csv('data_float.txt')
df.to_csv('data.csv', index = None)


df = pd.read_csv("{}".format(x),delimiter=",")
df.to_csv("{}".format(y))

NameError: name 'x' is not defined

2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [9]:
import json # import the JSON module
data = json.load(open('user_data.json'))
#print(data)
fdata = 
print(fdata)

with open('Data.csv', 'w') as file:
    [i for i in data if i["CreditCardType"] == "American Express"]
    writer = csv.writer(file)
    writer.writerow(fdata)
        

SyntaxError: invalid syntax (258740825.py, line 4)

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?